# Script to predict dynamic time stepping of an ocean model

Written by Rachel Furner, April 2019.

Collaborative work with colleagues at BAS, and the ATI



1. Read in the data
2. Define input data and output data.
3. Split into test and train data.
4. Train the network
5. Run predictions on test data and assess.

In [102]:
%matplotlib inline
import xarray as xr
import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

import pandas as pd
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import datetime
from os.path import join


import tensorflow as tf
#import gpflow
from gpflow.kernels import RBF, Periodic
from keras.models import Sequential

model = Sequential()

ModuleNotFoundError: No module named 'keras'

Define variables for this experiment

In [96]:
StepSize = 10 #number of time slices over which to predict

Read in data files as Xarrays

In [99]:
file = '/Users/rachelfurner/JasminData/DynPred/4500yr_Windx0.50/state.0000000000.t001.nc'
ds1   = xr.open_dataset(file)
ds1

<xarray.Dataset>
Dimensions:  (T: 2841, X: 11, Xp1: 12, Y: 78, Yp1: 79, Z: 42, Zl: 42)
Coordinates:
  * T        (T) float64 0.0 2.592e+06 5.184e+06 ... 7.359e+09 7.361e+09
  * Xp1      (Xp1) float64 0.0 2.0 4.0 6.0 8.0 10.0 ... 14.0 16.0 18.0 20.0 22.0
  * Y        (Y) float64 -59.5 -58.48 -57.44 -56.36 ... 58.48 59.5 60.5 61.5
  * Z        (Z) float64 -5.0 -15.05 -25.25 ... -4.375e+03 -4.625e+03 -4.875e+03
  * X        (X) float64 1.0 3.0 5.0 7.0 9.0 11.0 13.0 15.0 17.0 19.0 21.0
  * Yp1      (Yp1) float64 -60.0 -59.0 -57.97 -56.91 ... 59.0 60.0 61.0 62.0
  * Zl       (Zl) float64 0.0 -10.0 -20.1 -30.4 ... -4.25e+03 -4.5e+03 -4.75e+03
Data variables:
    iter     (T) int32 ...
    U        (T, Z, Y, Xp1) float32 ...
    V        (T, Z, Yp1, X) float32 ...
    Temp     (T, Z, Y, X) float32 ...
    S        (T, Z, Y, X) float32 ...
    Eta      (T, Y, X) float32 ...
    W        (T, Zl, Y, X) float32 ...
Attributes:
    the_run_name:    2deg
    MITgcm_version:  checkpoint67g
    build

###### Define Input output pairs
Inputs are full model field at t-SliceSize and t, outputs are full model fields at t+SliceSize.

SliceSize currently variable - plan to test different values and see how well things work.

Take sets of inputs and outputs from 30 steps apart, in bid to ensure some independance between samples.


In [98]:
#for s in range(0, int(max(ds1.T.data)), 30):

ds1_substepped_Tm1=ds1.isel(T=range(0,150,30))
ds1_substepped_T  =ds1.isel(T=range(StepSize,150+StepSize,30))
ds1_substepped_Tp1=ds1.isel(T=range(2*StepSize,150+2*StepSize,30))

feature_IDs = ['U', 'V', 'Temp', 'S', 'Eta', 'W']
Xtr=np.zeros((2*len(feature_IDs), ds1_substepped_T.T.size, ds1_substepped_T.X.size, ds1_substepped_T.Y.size, ds1_substepped_T.Z.size))
print(Xtr.shape)
ds1_substepped_T.U




#fill the thing!!
#Xtr[????]=????


(12, 5, 11, 78, 42)


<xarray.DataArray 'U' (T: 5, Z: 42, Y: 78, Xp1: 12)>
[196560 values with dtype=float32]
Coordinates:
  * T        (T) float64 2.592e+07 1.037e+08 1.814e+08 2.592e+08 3.37e+08
  * Xp1      (Xp1) float64 0.0 2.0 4.0 6.0 8.0 10.0 ... 14.0 16.0 18.0 20.0 22.0
  * Y        (Y) float64 -59.5 -58.48 -57.44 -56.36 ... 58.48 59.5 60.5 61.5
  * Z        (Z) float64 -5.0 -15.05 -25.25 ... -4.375e+03 -4.625e+03 -4.875e+03
Attributes:
    units:        m/s
    coordinates:  XU YU RC iter

Normalise data

In [ ]:
def normalise_data(train,test):
    ### Normalise training and test data 
    train_mean, train_std = np.mean(train), np.std(train)
    train = (train - train_mean) / train_std
    test  = (test - train_mean) / train_std
    return train, test